In [1]:
from deap import base, creator, tools
import random
import numpy as np
from sklearn.linear_model import LinearRegression, LassoLarsIC, Lasso
import pandas as pd

In [2]:
from algorithm.genetic_algorithm import read_data
X = read_data()
X

array([[3.17966504, 3.03062471, 3.22275227, ..., 3.70514307, 3.92179743,
        3.5667659 ],
       [3.31315547, 3.12753458, 3.36718872, ..., 3.00232446, 3.48703507,
        3.75358101],
       [3.5318474 , 3.55448902, 3.44163644, ..., 3.53159902, 3.77334177,
        3.25474063],
       ...,
       [3.55076635, 3.48014853, 3.65358394, ..., 2.69928466, 0.9630304 ,
        3.1252519 ],
       [3.64113611, 3.67916051, 3.13758092, ..., 4.34764293, 3.85532063,
        3.21161165],
       [3.58635986, 2.83222446, 3.24274907, ..., 2.69928466, 0.73012916,
        3.04170596]])

In [3]:
from algorithm.genetic_algorithm import create_individual, evaluate, mate, mutate

IND_SIZE = 1

# TODO SEED

# minimize fitness  
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# 2d list to represent DAG
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# creating population
toolbox.register("attribute", create_individual, X)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=IND_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# mate, mutate, select and evaluate functions
toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)  # TODO: look into selection functions
toolbox.register("evaluate", evaluate)



In [4]:
# TODO this is just copied from the docs:
def main():
    pop = toolbox.population(n=50)
    CXPB, MUTPB, NGEN = 0.5, 0.2, 40

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, pop)
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit

    for g in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = map(toolbox.clone, offspring)

        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

    return pop

In [5]:
pop = main()

[1 0]
(11672, 2)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 1 but corresponding boolean dimension is 11